In [1]:
import networkx as nx
from pyvis.network import Network
import os
import pandas as pd
from nltk.corpus import wordnet as wn

In [3]:
class FilteredSynset:
    def __init__(self, synset, score_dict=None, thresh=None):
        self.synset = synset
        self.score_dict = score_dict or {}
        self.thresh = thresh

    def name(self):
        return self.synset.name()

    def pos(self):
        return self.synset.pos()

    def definition(self):
        return self.synset.definition()

    def examples(self):
        return self.synset.examples()

    def offset(self):
        return self.synset.offset()

    def lemmas(self):
        return self.synset.lemmas()

    def hyponyms(self):
        result = []

        for child in self.synset.hyponyms():

            child_score = self.score_dict.get(f"n{child.offset():08d}", 0)

            filtered_child = FilteredSynset(child, self.score_dict, self.thresh)
            descendants = filtered_child.hyponyms()

            if (child_score >= self.thresh) or descendants:
                result.append(filtered_child)

        return result

In [20]:
df1 = pd.read_pickle("df1.pkl")
df1

,synset_id,description,score,synset_name,mapped_synset_list,mapped_synset_id,mapped_score,mapped_word_list
0,n01440764,"tench, Tinca tinca",0.00,tench.n.01,tench.n.01 → cyprinid.n.01 → cypriniform_fish....,n02512053,5.00,[fish]
1,n01443537,"goldfish, Carassius auratus",4.79,goldfish.n.01,goldfish.n.01 → cyprinid.n.01 → cypriniform_fi...,n02512053,5.00,[fish]
2,n01484850,"great white shark, white shark, man-eater, man...",0.00,great_white_shark.n.01,great_white_shark.n.01 → mackerel_shark.n.01 →...,n01482330,4.93,[shark]
3,n01491361,"tiger shark, Galeocerdo cuvieri",0.00,tiger_shark.n.01,tiger_shark.n.01 → requiem_shark.n.01 → shark....,n01482330,4.93,[shark]
4,n01494475,"hammerhead, hammerhead shark",4.28,hammerhead.n.03,hammerhead.n.03 → shark.n.01,n01482330,4.93,[shark]
...,...,...,...,...,...,...,...,...
995,n13044778,earthstar,0.00,earthstar.n.01,earthstar.n.01 → fungus.n.01 → organism.n.01 →...,None,NaN,NaN
996,n13052670,"hen-of-the-woods, hen of the woods, Polyporus ...",0.00,hen-of-the-woods.n.01,hen-of-the-woods.n.01 → fungus.n.01 → organism...,None,NaN,NaN
997,n13054560,bolete,0.00,bolete.n.01,bolete.n.01 → fungus.n.01 → organism.n.01 → li...,None,NaN,NaN
998,n13133613,"ear, spike, capitulum",5.00,ear.n.05,n13133613,n13133613,5.00,"[ear, spike, capitulum]"


In [18]:
df2 = pd.read_pickle("82k_synset_concreteness.pkl")
df2

,synset_id,words,synset_name,concreteness
0,n00001740,[entity],entity,2.68
1,n00001930,[physical entity],physical entity,0.00
2,n00002137,"[abstraction, abstract entity]",abstraction,1.57
3,n00002452,[thing],thing,3.17
4,n00002684,"[object, physical object]",object,3.66
...,...,...,...,...
82110,n15299225,[study hall],study hall,0.00
82111,n15299367,"[Transfiguration, Transfiguration Day, August 6]",transfiguration,2.35
82112,n15299585,[usance],usance,0.00
82113,n15299783,[window],window,4.86


In [ ]:
def visualize_hyponym_tree_interactive_stylish(
    root_synset_name, 
    score_col=None, 
    thresh=None, 
    max_depth=6, 
    max_nodes=300, 
    show_only_threshold=False, 
    output_file="figures/hyponym_tree_view.html"
):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    if isinstance(root_synset_name, (FilteredSynset, wn.synset.__class__)):
        root = root_synset_name
    elif isinstance(root_synset_name, str):
        root = wn.synset(root_synset_name)
    else:
        raise ValueError("Invalid root")

    if score_col is not None and hasattr(score_col, "to_dict"):
        score_col = score_col.to_dict()
    
    G = nx.DiGraph()
    net = Network(height="1500px", width="2000px", directed=True, bgcolor="#1E1E2F", font_color="#FFFFFF")
    net.from_nx(G)

    # Barnes-Hut physics: adjust for easy dragging and spacing
    net.barnes_hut(
        gravity=-200000,          # repulsion from center (negative = pushes nodes out)
        central_gravity=0.3,     # pulls nodes slightly toward center
        spring_length=200,       # desired length of edges
        spring_strength=0.05,    # stiffness of edges
        damping=0.09,            # slows oscillations
        overlap=0                # prevents nodes from overlapping
    )

    const_options = {
        "physics": {
            "barnesHut": {
            "theta": 0.9,
            "gravitationalConstant": -20250,
            "centralGravity": 0.25,
            "springLength": 200,
            "damping": 0.14
            },
            "maxVelocity": 191,
            "minVelocity": 0.75
        }
    }
    visited = set()
    node_count = 0



    CATEGORY_COLORS = {
        "animal.n.01": "#6935EA",
        "clothing.n.01": "#DF3CD1",
        "food.n.01": "#EEA25F",
        "food.n.02": "#EEA25F",
        "beverage.n.01": "#EEA25F",
        "implement.n.01": "#1EC35D",
        "device.n.01": "#1EC35D",
        "equipment.n.01": "#1EC35D",
        "durables.n.01": "#1EC35D",
        "conveyance.n.03": "#1E92C3",
    }

    def dfs(node, depth, inherited_color=None):
        nonlocal node_count
        if depth > max_depth or node_count >= max_nodes:
            return

        node_id = f"n{node.offset():08d}"
        if node_id in visited:
            return
        visited.add(node_id)

        score = score_col.get(node_id, 0) if score_col else 0
        label = node.name().replace("_", " ") + f"\n{score:.2f}"

        node_name = node.name()

        # activate or inherit category color
        current_color = CATEGORY_COLORS.get(node_name, inherited_color)


        # final size and color decision
        size = 2 ** score + 3

        if current_color:
            color = current_color
        elif depth == 0:
            color = "#FFFFFF"
            size = 50
        else:
            color = "#493F52"

  

        G.add_node(
            node_id,
            label=label,
            size=size,
            color=color,
            font={'size': 400, 'color': '#FFFFFF'}
        )
        node_count += 1

        for child in node.hyponyms():
            if node_count >= max_nodes:
                break
            if not isinstance(child, FilteredSynset):
                child = FilteredSynset(child, score_col, thresh)

            child_id = f"n{child.offset():08d}"
            child_score = score_col.get(child_id, 0) if score_col else 0
            if show_only_threshold and child_score < (thresh or 0):
                continue

            G.add_edge(node_id, child_id)
            dfs(child, depth + 1, current_color)


    dfs(root, 0,None)

    # Dark mode network
    net = Network(
        height="1500px",
        width="1500px",
        directed=True,
        bgcolor="#1E1E2F",  # dark background
        font_color="#FFFFFF"
    )
  
    net.from_nx(G)
    net.show_buttons(filter_=['physics'])
    net.write_html(output_file)

df1 = pd.read_pickle("df1.pkl")
score_dict = df1.set_index("mapped_synset_id")["mapped_score"].to_dict()
root_synset = wn.synset("entity.n.01")
filtered_root = FilteredSynset(root_synset, score_dict=score_dict, thresh=5.0)

visualize_hyponym_tree_interactive_stylish(
    filtered_root,
    score_col=score_dict,
    thresh=1.0,
    max_depth=100,       # keep small for testing
    max_nodes=100000,     # browser-safe
    show_only_threshold=False
)

In [27]:
df2 = pd.read_pickle("82k_synset_concreteness.pkl")
score_dict = df2.set_index("synset_id")["concreteness"].to_dict()
root_synset = wn.synset("entity.n.01")
filtered_root = FilteredSynset(root_synset, score_dict=score_dict, thresh=4.9)

visualize_hyponym_tree_interactive_stylish(
    filtered_root,
    score_col=score_dict,
    thresh=4.9,
    max_depth=12,       # keep small for testing
    max_nodes=100,     # browser-safe
    show_only_threshold=False
)

In [34]:
df1 = pd.read_pickle("df1.pkl")
score_dict = df1.set_index("mapped_synset_id")["mapped_score"].to_dict()
root_synset = wn.synset("entity.n.01")
filtered_root = FilteredSynset(root_synset, score_dict=score_dict, thresh=5.0)

visualize_hyponym_tree_interactive_stylish(
    filtered_root,
    score_col=score_dict,
    thresh=1.0,
    max_depth=100,       # keep small for testing
    max_nodes=100000,     # browser-safe
    show_only_threshold=False
)

UnboundLocalError: local variable 'active_color' referenced before assignment